This notebook cleans the datasets.

In [29]:
import pandas as pd
import numpy as np
import re
import itertools
import csv
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

pd.set_option('display.max_columns', 500)

train_data = pd.read_csv("../data/raw_train_data.csv", dtype={"zip_code" : object})
test_data = pd.read_csv("../data/raw_test_data.csv", dtype={"zip_code" : object})

### Cope with unrealistic data

Drop fraud = -1. Set annual_income = -1 and age > 100 to be missing values.

In [30]:
## fraud
train_data = train_data[train_data.fraud != -1]

## annual_income
train_data.loc[train_data.annual_income==-1, 'annual_income'] = np.nan
test_data.loc[test_data.annual_income==-1, 'annual_income'] = np.nan

## age_of_driver
train_data.loc[train_data.age_of_driver>100, 'age_of_driver'] = np.nan
test_data.loc[test_data.age_of_driver>100, 'age_of_driver'] = np.nan

### Encode Categorical Variables

Now, let's set the claim number as the index we are working from, and set marital status, higher education index, address change index, zip code, wintness indicator, policy report filed, and fraud to categorical variables.

In [31]:
# set claim_number as index:
train_data = train_data.set_index('claim_number')
test_data = test_data.set_index('claim_number')

train_data["marital_status"] = pd.Categorical(train_data["marital_status"])
train_data["high_education_ind"] = pd.Categorical(train_data["high_education_ind"])
train_data["address_change_ind"] = pd.Categorical(train_data["address_change_ind"])
train_data["zip_code"] = pd.Categorical(train_data["zip_code"])
train_data["witness_present_ind"] = pd.Categorical(train_data["witness_present_ind"])
train_data["policy_report_filed_ind"] = pd.Categorical(train_data["policy_report_filed_ind"])
train_data["fraud"] = pd.Categorical(train_data["fraud"])

test_data["marital_status"] = pd.Categorical(test_data["marital_status"])
test_data["high_education_ind"] = pd.Categorical(test_data["high_education_ind"])
test_data["address_change_ind"] = pd.Categorical(test_data["address_change_ind"])
test_data["zip_code"] = pd.Categorical(test_data["zip_code"])
test_data["witness_present_ind"] = pd.Categorical(test_data["witness_present_ind"])
test_data["policy_report_filed_ind"] = pd.Categorical(test_data["policy_report_filed_ind"])

Break the claim dates into year, month, and day of month as separate variables.

In [46]:
month_dict = {
    1 : 'January',
    2 : 'February',
    3 : 'March',
    4 : 'April',
    5 : 'May',
    6 : 'June',
    7 : 'July',
    8 : 'August',
    9 : 'September',
    10 : 'October',
    11 : 'November',
    12 : 'December'
}
### On the train data
train_data["claim_month"] = train_data['claim_date'].apply(
    lambda x: month_dict[int(re.search("^(\d+)/", x).group(1))]
)
train_data["claim_day"] = train_data['claim_date'].apply(
    lambda x: int(re.search("/(\d+)/", x).group(1))
)
train_data["claim_year"] = train_data['claim_date'].apply(
    lambda x: int(re.search("/(\d+)$", x).group(1))
)


### On the test data
test_data["claim_month"] = test_data['claim_date'].apply(
    lambda x: month_dict[int(re.search("^(\d+)/", x).group(1))]
)
test_data["claim_day"] = test_data['claim_date'].apply(
    lambda x: int(re.search("/(\d+)/", x).group(1))
)
test_data["claim_year"] = test_data['claim_date'].apply(
    lambda x: int(re.search("/(\d+)$", x).group(1))
)

## Add new features: Lat/Lon/State, economic indices

Downloaded zip code data from https://www.unitedstateszipcodes.org/zip-code-database/. And use zip code to get the latitude, longitude and state.

In [34]:
with open("../data/zip_code_database.csv", newline='') as csvfile:
    csv_reader = csv.DictReader(csvfile, delimiter=',')
    zip_to_lat = {}
    zip_to_lon = {}
    zip_to_state = {}
    for zip_data in csv_reader:
        zip_to_lat[zip_data['zip']] = float(zip_data['latitude'])
        zip_to_lon[zip_data['zip']] = float(zip_data['longitude'])
        zip_to_state[zip_data['zip']] = zip_data['state']
        
### assuming the '0' zip code is NaN (no such thing as a zip code of 0)   
zip_to_lat[np.nan] = np.nan
zip_to_lon[np.nan] = np.nan
zip_to_state[np.nan] = np.nan

zip_to_lat['0'] = np.nan
zip_to_lon['0'] = np.nan
zip_to_state['0'] = np.nan

### transform zip code to latitude, longitude, and state
latitude_train = train_data['zip_code'].apply(
    lambda x: zip_to_lat[x]
)
longitude_train = train_data['zip_code'].apply(
    lambda x: zip_to_lon[x]
)
state_train = train_data['zip_code'].apply(
    lambda x: zip_to_state[x]
)
latitude_train.name = 'latitude'
longitude_train.name = 'longitude'
state_train.name = 'state'

latitude_test = test_data['zip_code'].apply(
    lambda x: zip_to_lat[x]
)
longitude_test = test_data['zip_code'].apply(
    lambda x: zip_to_lon[x]
)
state_test = test_data['zip_code'].apply(
    lambda x: zip_to_state[x]
)
latitude_test.name = 'latitude'
longitude_test.name = 'longitude'
state_test.name = 'state'


### Add these new features to the data frame
train_data = pd.concat([train_data, latitude_train], axis=1)
train_data = pd.concat([train_data, longitude_train], axis=1)
train_data = pd.concat([train_data, state_train], axis=1)
train_data["state"] = pd.Categorical(train_data["state"])

test_data = pd.concat([test_data, latitude_test], axis=1)
test_data = pd.concat([test_data, longitude_test], axis=1)
test_data = pd.concat([test_data, state_test], axis=1)
test_data["state"] = pd.Categorical(test_data["state"])

Load economic data and put into dataset.

In [35]:
unemp_data = pd.read_pickle('../data/unemployment_data/unemp_data.pkl')

# For train data
claim_month = train_data['claim_month'].values
claim_state = train_data['state'].values
claim_year = train_data['claim_year'].values

unemp_rate = []
for x in zip(claim_month, claim_state, claim_year): 
    unemp_rate_lookup = unemp_data[(unemp_data['month']==x[0])
                                   & (unemp_data['state']==x[1])
                                   & (unemp_data['year']==x[2])]['unemp_rate'].values
    if not unemp_rate_lookup:
        unemp_rate_lookup = [np.nan]
    unemp_rate.append(unemp_rate_lookup[0])
train_data['unemp_rate'] = unemp_rate


# For test data
claim_month = test_data['claim_month'].values
claim_state = test_data['state'].values
claim_year = test_data['claim_year'].values

unemp_rate = []
for x in zip(claim_month, claim_state, claim_year): 
    unemp_rate_lookup = unemp_data[(unemp_data['month']==x[0])
                                   & (unemp_data['state']==x[1])
                                   & (unemp_data['year']==x[2])]['unemp_rate'].values
    if not unemp_rate_lookup:
        unemp_rate_lookup = [np.nan]
    unemp_rate.append(unemp_rate_lookup[0])
test_data['unemp_rate'] = unemp_rate

## Imputation of Missing Values

Use mode and mean to impute.

In [37]:
# mean of age_of_driver
age_of_driver_mean = train_data.age_of_driver.mean()
train_data['age_of_driver'].fillna(age_of_driver_mean, inplace=True)
test_data['age_of_driver'].fillna(age_of_driver_mean, inplace=True)

# mode of marital_status
marital_status_mode = train_data.marital_status.mode().values[0]
train_data['marital_status'].fillna(marital_status_mode, inplace=True)
test_data['marital_status'].fillna(marital_status_mode, inplace=True)

# average of annual_income
annual_income_mean = train_data.annual_income.mean()
train_data['annual_income'].fillna(annual_income_mean, inplace=True)
test_data['annual_income'].fillna(annual_income_mean, inplace=True)

# mode of witness_present_ind
witness_present_mode = train_data.witness_present_ind.mode().values[0]
train_data['witness_present_ind'].fillna(witness_present_mode, inplace=True)
test_data['witness_present_ind'].fillna(witness_present_mode, inplace=True)

# mean of claim_est_payout
claim_est_payout_mean = train_data.claim_est_payout.mean()
train_data['claim_est_payout'].fillna(claim_est_payout_mean, inplace=True)
test_data['claim_est_payout'].fillna(claim_est_payout_mean, inplace=True)

# mean of age_of_vehicle
age_of_vehicle_mean = train_data.age_of_vehicle.mean()
train_data['age_of_vehicle'].fillna(age_of_vehicle_mean, inplace=True)
test_data['age_of_vehicle'].fillna(age_of_vehicle_mean, inplace=True)

# mean latitude
latitude_mean = train_data.latitude.mean()
train_data['latitude'].fillna(latitude_mean, inplace=True)
test_data['latitude'].fillna(latitude_mean, inplace=True)

# mean longitude
longitude_mean = train_data.longitude.mean()
train_data['longitude'].fillna(longitude_mean, inplace=True)
test_data['longitude'].fillna(longitude_mean, inplace=True)

# mode of state
state_mode = train_data.state.mode().values[0]
train_data['state'].fillna(state_mode, inplace=True)
test_data['state'].fillna(state_mode, inplace=True)

# mean of unemp_rate
unemp_rate_mean = train_data.unemp_rate.mean()
train_data['unemp_rate'].fillna(unemp_rate_mean, inplace=True)
test_data['unemp_rate'].fillna(unemp_rate_mean, inplace=True)

# print the list of missing columns
print(list(itertools.compress(list(train_data), list(train_data.isna().any()))))
print(list(itertools.compress(list(test_data), list(test_data.isna().any()))))

[]
[]


## One-Hot Encoding of Categorical Variables

Looks like we've replaced all na values in testing and training.  The next thing I want to do is encode categorical variables that have more than two categories using one hot encoding.  I first check that the same number of categories exist in both the training and testing data sets:

In [38]:
###encoding for TRAIN data set

# one-hot encoding for day of week
day_dummies = pd.get_dummies(train_data['claim_day_of_week'], 
                             prefix='claim_day', drop_first=True)
train_data = pd.concat([train_data, day_dummies], axis=1)
train_data.drop(["claim_day_of_week"], axis=1, inplace=True)

# one-hot encoding for site of accident
accident_dummies = pd.get_dummies(train_data['accident_site'], 
                                  prefix='accident_site', drop_first=True)
train_data = pd.concat([train_data, accident_dummies], axis=1)
train_data.drop(["accident_site"], axis=1, inplace=True)

# one-hot encoding for channel
channel_dummies = pd.get_dummies(train_data['channel'], 
                                 prefix='channel', drop_first=True)
train_data = pd.concat([train_data, channel_dummies], axis=1)
train_data.drop(["channel"], axis=1, inplace=True)

# one-hot encoding for vehicle category
vehicle_cat_dummies = pd.get_dummies(train_data['vehicle_category'], 
                                 prefix='vehicle_category', drop_first=True)
train_data = pd.concat([train_data, vehicle_cat_dummies], axis=1)
train_data.drop(["vehicle_category"], axis=1, inplace=True)

# one-hot encoding for vehicle color
vehicle_color_dummies = pd.get_dummies(train_data['vehicle_color'], 
                                 prefix='vehicle_color', drop_first=True)
train_data = pd.concat([train_data, vehicle_color_dummies], axis=1)
train_data.drop(["vehicle_color"], axis=1, inplace=True)

# one-hot encoding for claim month
vehicle_color_dummies = pd.get_dummies(train_data['claim_month'], 
                                 prefix='claim_month', drop_first=True)
train_data = pd.concat([train_data, vehicle_color_dummies], axis=1)
train_data.drop(["claim_month"], axis=1, inplace=True)

# one-hot encoding for state
state_dummies = pd.get_dummies(train_data['state'],
                               prefix='state', drop_first=True)
train_data = pd.concat([train_data, state_dummies], axis=1)
train_data.drop(["state"], axis=1, inplace=True)



### encoding for TEST data set

# one-hot encoding for day of week
day_dummies = pd.get_dummies(test_data['claim_day_of_week'], 
                             prefix='claim_day', drop_first=True)
test_data = pd.concat([test_data, day_dummies], axis=1)
test_data.drop(["claim_day_of_week"], axis=1, inplace=True)

# one-hot encoding for site of accident
accident_dummies = pd.get_dummies(test_data['accident_site'], 
                                  prefix='accident_site', drop_first=True)
test_data = pd.concat([test_data, accident_dummies], axis=1)
test_data.drop(["accident_site"], axis=1, inplace=True)

# one-hot encoding for channel
channel_dummies = pd.get_dummies(test_data['channel'], 
                                 prefix='channel', drop_first=True)
test_data = pd.concat([test_data, channel_dummies], axis=1)
test_data.drop(["channel"], axis=1, inplace=True)

# one-hot encoding for vehicle category
vehicle_cat_dummies = pd.get_dummies(test_data['vehicle_category'], 
                                 prefix='vehicle_category', drop_first=True)
test_data = pd.concat([test_data, vehicle_cat_dummies], axis=1)
test_data.drop(["vehicle_category"], axis=1, inplace=True)

# one-hot encoding for vehicle color
vehicle_color_dummies = pd.get_dummies(test_data['vehicle_color'], 
                                 prefix='vehicle_color', drop_first=True)
test_data = pd.concat([test_data, vehicle_color_dummies], axis=1)
test_data.drop(["vehicle_color"], axis=1, inplace=True)

# one-hot encoding for claim month
vehicle_color_dummies = pd.get_dummies(test_data['claim_month'], 
                                 prefix='claim_month', drop_first=True)
test_data = pd.concat([test_data, vehicle_color_dummies], axis=1)
test_data.drop(["claim_month"], axis=1, inplace=True)

# one-hot encoding for state
state_dummies = pd.get_dummies(test_data['state'],
                               prefix='state', drop_first=True)
test_data = pd.concat([test_data, state_dummies], axis=1)
test_data.drop(["state"], axis=1, inplace=True)

In [39]:
### clean up variable names by making them all lowercase with underscore separators.
train_data.columns = map(
    lambda s: s.lower().replace(' ', '_'), 
    train_data.columns
)

test_data.columns = map(
    lambda s: s.lower().replace(' ', '_'), 
    test_data.columns
)


### Add interest rate

In [ ]:
Int=pd.read_csv('data/Interest_rate.csv')  ## the dataset is a little bit different from the original one. Check later.

month_dict = {
  1 : 'M01', 2 : 'M02', 3 : 'M03', 4 : 'M04', 5 : 'M05',6 : 'M06',7 : 'M07',8 : 'M08',9 : 'M09',10 : 'M10', 11 : 'M11',12 : 'M12'
}

for key in month_dict:
    train_data.loc[(train_data['claim_month']==month_dict[key])&(train_data['claim_year']==2015),'Interest_rate']=Int.loc[key-1,'Interest_rate']
    train_data.loc[(train_data['claim_month']==month_dict[key])&(train_data['claim_year']==2016),'Interest_rate']=Int.loc[key+11,'Interest_rate']
    test_data.loc[(test_data['claim_month']==month_dict[key])&(test_data['claim_year']==2015),'Interest_rate']=Int.loc[key-1,'Interest_rate']
    test_data.loc[(test_data['claim_month']==month_dict[key])&(test_data['claim_year']==2016),'Interest_rate']=Int.loc[key+11,'Interest_rate']

### Output dataset

In [40]:
train_data.to_csv('../data/train_data_clean.csv', header=True)
test_data.to_csv('../data/test_data_clean.csv', header=True)

Due to the loss of some original code and data, some details are not included in this notebook. And the final data cleaning file is "../data/train_data_clean3.csv"

### Add grouped-by means

In [54]:
## read data
train = pd.read_csv("../data/train_data_clean3.csv")
test = pd.read_csv("../data/test_data_clean3.csv")
raw_train = pd.read_csv('../data/raw_train_data.csv')
raw_test = pd.read_csv('../data/raw_test_data.csv')


## add Unemployment data
df1 = pd.read_csv("../data/train_data_clean2.csv")
train['Unem_rate'] = df1['Unem_rate']
df2 = pd.read_csv('../data/test_data_clean2.csv')
test['Unem_rate'] = df2['Unem_rate']

## add interest data
train_interest = pd.read_csv("../data/train_data_interest.csv")
train['Interest_rate'] = train_interest['Interest_rate']
test_interest = pd.read_csv("../data/test_data_interest.csv")
test['Interest_rate'] = test_interest['Interest_rate']


## gender
grouped_gender = train["fraud"].groupby(train['gender'])
grouped_gender_mean = grouped_gender.mean().to_frame()
grouped_gender_mean['gender']=grouped_gender_mean.index
grouped_gender_mean['fraud_gender'] = grouped_gender_mean['fraud']
grouped_gender_mean.drop('fraud', axis = 1, inplace = True)
train = pd.merge(train, grouped_gender_mean, on = "gender", how = "left")
test = pd.merge(test, grouped_gender_mean, on = "gender", how = "left")
grouped_gender_mean

## marital_status
grouped_marital_status = train["fraud"].groupby(train['marital_status'])
grouped_marital_status_mean = grouped_marital_status.mean().to_frame()
grouped_marital_status_mean['marital_status']=grouped_marital_status_mean.index
grouped_marital_status_mean['fraud_marital_status'] = grouped_marital_status_mean['fraud']
grouped_marital_status_mean.drop('fraud', axis = 1, inplace = True)
train = pd.merge(train, grouped_marital_status_mean, on = "marital_status", how = "left")
test = pd.merge(test, grouped_marital_status_mean, on = "marital_status", how = "left")
grouped_marital_status_mean

## high_education_ind
grouped_high_education_ind = train["fraud"].groupby(train['high_education_ind'])
grouped_high_education_ind_mean = grouped_high_education_ind.mean().to_frame()
grouped_high_education_ind_mean['high_education_ind']=grouped_high_education_ind_mean.index
grouped_high_education_ind_mean['fraud_high_education_ind'] = grouped_high_education_ind_mean['fraud']
grouped_high_education_ind_mean.drop('fraud', axis = 1, inplace = True)
train = pd.merge(train, grouped_high_education_ind_mean, on = "high_education_ind", how = "left")
test = pd.merge(test, grouped_high_education_ind_mean, on = "high_education_ind", how = "left")
grouped_high_education_ind_mean

## address_change_ind
grouped_address_change_ind = train["fraud"].groupby(train['address_change_ind'])
grouped_address_change_ind_mean = grouped_address_change_ind.mean().to_frame()
grouped_address_change_ind_mean['address_change_ind']=grouped_address_change_ind_mean.index
grouped_address_change_ind_mean['fraud_address_change_ind'] = grouped_address_change_ind_mean['fraud']
grouped_address_change_ind_mean.drop('fraud', axis = 1, inplace = True)
train = pd.merge(train, grouped_address_change_ind_mean, on = "address_change_ind", how = "left")
test = pd.merge(test, grouped_address_change_ind_mean, on = "address_change_ind", how = "left")
grouped_address_change_ind_mean

## living_status
grouped_living_status = train["fraud"].groupby(train['living_status'])
grouped_living_status_mean = grouped_living_status.mean().to_frame()
grouped_living_status_mean['living_status']=grouped_living_status_mean.index
grouped_living_status_mean['fraud_living_status'] = grouped_living_status_mean['fraud']
grouped_living_status_mean.drop('fraud', axis = 1, inplace = True)
train = pd.merge(train, grouped_living_status_mean, on = "living_status", how = "left")
test = pd.merge(test, grouped_living_status_mean, on = "living_status", how = "left")
grouped_living_status_mean

## zip_code
grouped_zip_code = train["fraud"].groupby(train['zip_code'])
grouped_zip_code_mean = grouped_zip_code.mean().to_frame()
grouped_zip_code_mean['zip_code']=grouped_zip_code_mean.index
grouped_zip_code_mean['fraud_zip_code'] = grouped_zip_code_mean['fraud']
grouped_zip_code_mean.drop('fraud', axis = 1, inplace = True)
train = pd.merge(train, grouped_zip_code_mean, on = "zip_code", how = "left")
test = pd.merge(test, grouped_zip_code_mean, on = "zip_code", how = "left")
grouped_zip_code_mean

## claim_date
grouped_claim_date = train["fraud"].groupby(train['claim_date'])
grouped_claim_date_mean = grouped_claim_date.mean().to_frame()
grouped_claim_date_mean['claim_date']=grouped_claim_date_mean.index
grouped_claim_date_mean['fraud_claim_date'] = grouped_claim_date_mean['fraud']
grouped_claim_date_mean.drop('fraud', axis = 1, inplace = True)
train = pd.merge(train, grouped_claim_date_mean, on = "claim_date", how = "left")
test = pd.merge(test, grouped_claim_date_mean, on = "claim_date", how = "left")
grouped_claim_date_mean

## witness_present_ind
grouped_witness_present_ind = train["fraud"].groupby(train['witness_present_ind'])
grouped_witness_present_ind_mean = grouped_witness_present_ind.mean().to_frame()
grouped_witness_present_ind_mean['witness_present_ind']=grouped_witness_present_ind_mean.index
grouped_witness_present_ind_mean['fraud_witness_present_ind'] = grouped_witness_present_ind_mean['fraud']
grouped_witness_present_ind_mean.drop('fraud', axis = 1, inplace = True)
train = pd.merge(train, grouped_witness_present_ind_mean, on = "witness_present_ind", how = "left")
test = pd.merge(test, grouped_witness_present_ind_mean, on = "witness_present_ind", how = "left")
grouped_witness_present_ind_mean

## policy_report_filed_ind
grouped_policy_report_filed_ind = train["fraud"].groupby(train['policy_report_filed_ind'])
grouped_policy_report_filed_ind_mean = grouped_policy_report_filed_ind.mean().to_frame()
grouped_policy_report_filed_ind_mean['policy_report_filed_ind']=grouped_policy_report_filed_ind_mean.index
grouped_policy_report_filed_ind_mean['fraud_policy_report_filed_ind'] = grouped_policy_report_filed_ind_mean['fraud']
grouped_policy_report_filed_ind_mean.drop('fraud', axis = 1, inplace = True)
train = pd.merge(train, grouped_policy_report_filed_ind_mean, on = "policy_report_filed_ind", how = "left")
test = pd.merge(test, grouped_policy_report_filed_ind_mean, on = "policy_report_filed_ind", how = "left")
grouped_policy_report_filed_ind_mean

## state
grouped_state = train["fraud"].groupby(train['state'])
grouped_state_mean = grouped_state.mean().to_frame()
grouped_state_mean['state']=grouped_state_mean.index
grouped_state_mean['fraud_state'] = grouped_state_mean['fraud']
grouped_state_mean.drop('fraud', axis = 1, inplace = True)
train = pd.merge(train, grouped_state_mean, on = "state", how = "left")
test = pd.merge(test, grouped_state_mean, on = "state", how = "left")
grouped_state_mean

## accident_site
grouped_accident_site = raw_train["fraud"].groupby(raw_train['accident_site'])
grouped_accident_site_mean = grouped_accident_site.mean().to_frame()
grouped_accident_site_mean['accident_site']=grouped_accident_site_mean.index
grouped_accident_site_mean['fraud_accident_site'] = grouped_accident_site_mean['fraud']
grouped_accident_site_mean.drop('fraud', axis = 1, inplace = True)
raw_train = pd.merge(raw_train, grouped_accident_site_mean, on = "accident_site", how = "left")
train['fraud_accident_site'] = raw_train['fraud_accident_site']
raw_test = pd.merge(raw_test, grouped_accident_site_mean, on = "accident_site", how = "left")
test['fraud_accident_site'] = raw_test['fraud_accident_site']
grouped_accident_site_mean

## channel
grouped_channel = raw_train["fraud"].groupby(raw_train['channel'])
grouped_channel_mean = grouped_channel.mean().to_frame()
grouped_channel_mean['channel']=grouped_channel_mean.index
grouped_channel_mean['fraud_channel'] = grouped_channel_mean['fraud']
grouped_channel_mean.drop('fraud', axis = 1, inplace = True)
raw_train = pd.merge(raw_train, grouped_channel_mean, on = "channel", how = "left")
train['fraud_channel'] = raw_train['fraud_channel']
raw_test = pd.merge(raw_test, grouped_channel_mean, on = "channel", how = "left")
test['fraud_channel'] = raw_test['fraud_channel']
              
grouped_channel_mean

## vehicle_category
grouped_vehicle_category = raw_train["fraud"].groupby(raw_train['vehicle_category'])
grouped_vehicle_category_mean = grouped_vehicle_category.mean().to_frame()
grouped_vehicle_category_mean['vehicle_category']=grouped_vehicle_category_mean.index
grouped_vehicle_category_mean['fraud_vehicle_category'] = grouped_vehicle_category_mean['fraud']
grouped_vehicle_category_mean.drop('fraud', axis = 1, inplace = True)
raw_train = pd.merge(raw_train, grouped_vehicle_category_mean, on = "vehicle_category", how = "left")
train['fraud_vehicle_category'] = raw_train['fraud_vehicle_category']
raw_test = pd.merge(raw_test, grouped_vehicle_category_mean, on = "vehicle_category", how = "left")
test['fraud_vehicle_category'] = raw_test['fraud_vehicle_category']              
grouped_vehicle_category_mean

## vehicle_color
grouped_vehicle_color = raw_train["fraud"].groupby(raw_train['vehicle_color'])
grouped_vehicle_color_mean = grouped_vehicle_color.mean().to_frame()
grouped_vehicle_color_mean['vehicle_color']=grouped_vehicle_color_mean.index
grouped_vehicle_color_mean['fraud_vehicle_color'] = grouped_vehicle_color_mean['fraud']
grouped_vehicle_color_mean.drop('fraud', axis = 1, inplace = True)
raw_train = pd.merge(raw_train, grouped_vehicle_color_mean, on = "vehicle_color", how = "left")
train['fraud_vehicle_color'] = raw_train['fraud_vehicle_color']
raw_test = pd.merge(raw_test, grouped_vehicle_color_mean, on = "vehicle_color", how = "left")
test['fraud_vehicle_color'] = raw_test['fraud_vehicle_color']              
grouped_vehicle_color_mean

,vehicle_color,fraud_vehicle_color
vehicle_color,,
black,black,0.162977
blue,blue,0.153454
gray,gray,0.154753
other,other,0.154274
red,red,0.157505
silver,silver,0.159221
white,white,0.151644


The thing need to note is that the above code will cause problems of overfitting and that is why we delete all of the grouped by means in the final model.

In [55]:
train.to_csv('../data/train_data_clean_4_grouped.csv', index = False)
test.to_csv('../data/test_data_clean_4_grouped.csv', index = False)